# Conversion of Raw lidar data to cloud and precipitation classification

## Abstract

This Notebook contains the post processing of the lidar data, including the spatial and temporal integration, the system calibration, background correction, SNR estimation, volume depolarization estimation, Cloud masking and hidrometeor clasification. Quick looks and some statistics are generated as well. Results are saved in netCDF format.

## Import libraries

Generic and own packages stored in "lib" are loaded to be used along the notebook. 

In [ ]:
import sys, os
sys.path.append("lib") # adding "lib" path with own packages
from Sigma_mol import sigma_mol # reads the radio sounsing and compute extinction coefficient
from scipy.interpolate import interp1d # to interpolate modeled variables to lidar heights
from lidar_integrating_space_time import Lidar_space_time as lidar_integ #integrates lidar raw data in height and time
from lidar_integrating_space_time_std import Lidar_space_time2 as lidar_integ_std #maybe not used
from fft_denoising import fft_denoising #maybe not used
from klett import Klett81b #maybe not used
import numpy as np 
import pylab #plots
from DP_simp import DP_simp # Curve simplification
from running_mean import running_mean # runing mean
from time2epoch import time2epoch #maybe not used?
#from cloud_mask_v1 import cloud_mask
from netCDF4 import Dataset 
#from time import sleep
from scipy import stats
from scipy.optimize import curve_fit
from dplots import densplot # make 2D desity plots
from Comb_LidarMRR import Comb_LidarMRR3 as Comb_LidarMRR 
from cloud_mask_v2 import cloud_mask2
from sm_paramTOP import sm_paramTOP
import matplotlib
from copy import copy

## Load MRR and other parameters


In [ ]:
#####Load MRR Data
path_MRR = "I:/PHD/Lidar/Inversion_V2/MRR_Data/"
Ze = np.loadtxt(path_MRR + "Ze_10min.txt")
times_MRR = np.loadtxt(path_MRR + "times.txt")
Height_MRR = np.loadtxt(path_MRR + "Height.txt")

Zem = np.ma.masked_where(Ze == -9999, Ze)

#####Load font format

font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 20}
        
pylab.rc('font', **font)      

#####Color Parameters
cmap = pylab.cm.jet
bounds = np.linspace(1,3,4)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
bounds2 = np.linspace(0,12,14)
norm2 = matplotlib.colors.BoundaryNorm(bounds2, cmap.N)

cmap2 = pylab.cm.get_cmap(cmap.name,8)